In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return Variable(tensor)

training_data = [
    ("The dog are the apple".split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ("Everybody read that book".split(), ['NN', 'V', 'DET', 'NN'])
]

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

tag_to_ix = { 'DET' : 0, 'NN' : 1, 'V' : 2 }

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'are': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [3]:
# here we implement the LSTM POS Tagger with word level LSTM as well as _char_ level LSTM
import string
char_to_ix = \
    dict([(w, i) for i, w in enumerate(string.ascii_lowercase)] + \
         [(w, i) for i, w in enumerate(string.ascii_uppercase)])

class BetterLSTMTagger(nn.Module):
    # here we assume the embedding dimension size and hidden dimension size are the same 
    # for both char and word level LSTM. It is easy to extend this to a more general case.
    def __init__(self, embedding_dim, hidden_dim, word_to_ix, char_to_ix, tag_to_ix):
        super(BetterLSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.char_embedding = nn.Embedding(len(char_to_ix), embedding_dim)
        self.word_embedding = nn.Embedding(len(word_to_ix), embedding_dim)
        self.char_lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.word_lstm = nn.LSTM(2 * embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, len(tag_to_ix))
        self.word_hidden = self.init_hidden()
        self.char_hidden = self.init_hidden()
        self.word_to_ix = word_to_ix
        self.char_to_ix = char_to_ix
    def init_hidden(self):
        return [ Variable(torch.zeros(1, 1, self.hidden_dim)), Variable(torch.zeros(1, 1, self.hidden_dim)) ]
    def prepare_word_inputs(self, sentence):
        return prepare_sequence(sentence, self.word_to_ix)
    def prepare_char_inputs(self, word):
        return prepare_sequence(word, self.char_to_ix)
    def forward_word(self, word):
        self.char_hidden = self.init_hidden()
        _, (output, _) = \
            self.char_lstm(
                self.char_embedding(
                    self.prepare_char_inputs(word)
                ).view(len(word), 1, -1), self.char_hidden)
        return output
    def forward(self, sentence):
        char_inputs = torch.cat([self.forward_word(word) for word in sentence], dim=0)
        word_inputs = self.word_embedding(self.prepare_word_inputs(sentence)).view(len(sentence), 1, -1)
        inputs = torch.cat([char_inputs, word_inputs], dim=2)
        out, self.word_hidden = self.word_lstm(inputs, self.word_hidden)
        tag_space = self.hidden2tag(out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    
tagger = BetterLSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, word_to_ix, char_to_ix, tag_to_ix)
tag_scores = tagger.forward(training_data[0][0])
print(tag_scores)

Variable containing:
-1.4077 -0.7995 -1.1850
-1.4793 -0.8146 -1.1105
-1.4641 -0.8211 -1.1125
-1.5262 -0.8029 -1.0948
-1.4237 -0.8239 -1.1380
[torch.FloatTensor of size 5x3]



In [4]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(tagger.parameters(), lr=0.1)

for epoch in range(300):
    loss_val = 0
    for sentence, tags in training_data:
        tagger.zero_grad()
        tagger.word_hidden = tagger.init_hidden()
        tag_scores = tagger(sentence)
        targets = prepare_sequence(tags, tag_to_ix)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        loss_val += loss.data
    if (epoch + 1) % 50 == 0:
        print('Epoch [%d/%d], Loss %.3f' % (epoch + 1, 300, loss_val))

Epoch [50/300], Loss 1.890
Epoch [100/300], Loss 1.208
Epoch [150/300], Loss 0.368
Epoch [200/300], Loss 0.149
Epoch [250/300], Loss 0.084
Epoch [300/300], Loss 0.057


In [5]:
from six import iteritems
import numpy as np

tag_scores = tagger(training_data[0][0])
print(tag_scores)

def reverse_lookup_seq(seq, to_ix):
    def lookup(j): return next(w for w, i in iteritems(to_ix) if j == i)
    return [lookup(w) for w in seq]
print('predicted tags')
# here shows a subtle yet big difference between pytorch and tensorflow
# while every operation in tensorflow can be characterized by a Tensor in tf
# pytorch only considers operations that is _differentiable__
# since argmax is not differentiable (or you may say it is 0 everywhere, so not really useful)
# pytorch does not offer an operation to do `argmax` directly
print(reverse_lookup_seq(np.argmax(tag_scores.data, axis=1), tag_to_ix))
print('target tags')
print(training_data[0][-1])

Variable containing:
-0.0739 -2.7282 -5.1311
-5.3814 -0.0099 -5.2579
-3.8599 -3.6682 -0.0477
-0.0342 -4.5396 -3.7739
-4.5373 -0.0308 -3.9313
[torch.FloatTensor of size 5x3]

predicted tags
['DET', 'NN', 'V', 'DET', 'NN']
target tags
['DET', 'NN', 'V', 'DET', 'NN']
